In [207]:
import tensorflow as tf
import numpy as np
import os
import time

In [208]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [209]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [210]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [211]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


In [212]:
example_texts = ['abcdefg', 'xyz']
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [213]:
ids_from_chars = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)

In [214]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [215]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [216]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [217]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [218]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [219]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [220]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [221]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [222]:
seq_length = 100

In [223]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [224]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [225]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [226]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [227]:
dataset = sequences.map(split_input_target)

In [228]:
for input_example,target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


# Membuat Batch Training

In [229]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

# Membuat Model

In [230]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [231]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

**Menambahkan Tracing Gradient**

In [232]:
class CustomTraining(MyModel):
    @tf.function
    def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))
      return {'loss': loss}

In [233]:
model = CustomTraining(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)


# Uji Model

In [234]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [235]:
model.summary()

Model: "custom_training_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     multiple                  16896     
                                                                 
 gru_4 (GRU)                 multiple                  3938304   
                                                                 
 dense_4 (Dense)             multiple                  67650     
                                                                 
Total params: 4,022,850
Trainable params: 4,022,850
Non-trainable params: 0
_________________________________________________________________


In [236]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [237]:
sampled_indices

array([56, 59, 48, 17, 48,  0, 53, 20, 49, 52, 52, 31, 11, 61, 17,  6, 31,
       62, 38, 63, 58, 23, 31, 61, 52, 17, 41, 42, 51, 61, 15, 43, 47, 33,
        9, 43, 51, 54, 26, 38, 34, 57, 21, 62, 23, 47, 17, 46, 49, 59,  9,
       22, 32, 61, 49, 40, 31, 17, 28,  0, 56, 45, 15, 46, 29, 33, 28, 41,
        6, 20, 46,  0, 49, 45, 58, 40, 65, 29, 39, 26, 53, 48, 21,  7,  5,
        8, 34,  4, 12, 62, 35, 17, 27, 16, 34, 35, 41, 25, 61, 41])

In [238]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'and seen the spider.\nCamillo was his help in this, his pander:\nThere is a plot against my life, my c'

Next Char Predictions:
 b"qtiDi[UNK]nGjmmR:vD'RwYxsJRvmDbclvBdhT.dloMYUrHwJhDgjt.ISvjaRDO[UNK]qfBgPTOb'Gg[UNK]jfsazPZMniH,&-U$;wVDNCUVbLvb"


# Train Model

**Optimizer dan fungsi loss**

In [239]:
loss =tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [240]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1896663, shape=(), dtype=float32)


In [241]:
tf.exp(example_batch_mean_loss).numpy()

66.00076

In [242]:
model.compile(optimizer='adam', loss=loss)

**Konfigurasi checkpoints**

In [243]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

**Proses Training**

In [244]:
EPOCHS = 20

In [245]:
mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()
    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])
        
        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)
            
    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))
        
    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)
    
model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 4.1897
Epoch 1 Batch 50 Loss 2.8407
Epoch 1 Batch 100 Loss 2.3468
Epoch 1 Batch 150 Loss 2.2092

Epoch 1 Loss: 2.7226
Time taken for 1 epoch 10.26 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 2.1774
Epoch 2 Batch 50 Loss 2.0504
Epoch 2 Batch 100 Loss 1.9664
Epoch 2 Batch 150 Loss 1.8622

Epoch 2 Loss: 1.9821
Time taken for 1 epoch 8.02 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.7723
Epoch 3 Batch 50 Loss 1.7262
Epoch 3 Batch 100 Loss 1.6530
Epoch 3 Batch 150 Loss 1.6804

Epoch 3 Loss: 1.7052
Time taken for 1 epoch 8.12 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.6185
Epoch 4 Batch 50 Loss 1.6184
Epoch 4 Batch 100 Loss 1.5424
Epoch 4 Batch 150 Loss 1.5494

Epoch 4 Loss: 1.5467
Time taken for 1 epoch 7.94 sec
________________________________________________________________________

In [246]:
# history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

## Generate Teks

In [247]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature =temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states, return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [248]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [249]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
Tybalt, the fairest friend;
What you is state of love;
Perhaps they say; we have appeared for a mile,
I thank your life approach this deeds do wind.

VOLUMNIA:
I pray, sir, what's the matter?

Messenger:
My crambril will not peace. Fireh, now, I tell you, he
is oath?

CORIOLANUS:
I had rather sleep o' the sing: why, how now of Jades!

DERBY:
As, thou wilt work another good.
Must I stary man. I must say to Rome,
He sends you there, what are you, great must be glad to dream
That which is more talk of our streets in stops
Except his majesty.

ISABELLA:
I am sorry, sitting, our general make his master.
nearing that comfort in this custom, sir?
Have you my fall, she hath a pasting flee
As day does raise thee so in me;
Thou rass, their fury, shall the world by life,
And prays healthy as an idla world. My faits be edwasted
Than 'dwax in thee in some debt of under the
hool. Admouad their swords can do me man,
With all bere which was well was burst, his blood
Will crave the time to come.

In [250]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nWhat's that, in men, they say is losing then to speak.\n\nHORTENSIO:\nFellors, he on mine hath commanded is all: I\nwould have fair, and my invetted Claudio's years?\n\nJOHN OF GAUNT:\nWhat is't o'er-vanive men, you wert oversedvantage;\nThey call him frail that slender state with Thomas,\nEmperient plucker than my noble cousin's dempetation.\n\nQUEEN MARGARET:\nTybalt, Mercutio, the father bears none.\n\nQUEEN ELIZABETH:\nA sun to come; this is a man doth choke our reason.\nThis is the cause of my mother's sake.\n\nQUEEN MARGARET:\nThou shalt revelg in all touch'd,\nAnd so my merry with so strong and feet\nWere in preserved a swails they call them peace.\nWelcome, good heart! O Podrection,\nWho buy'st to make they well avoudded foul mouths, to stay you\nDew'd. Pardon me, good father.\n\nPETRUCHIO:\nThou call'dst me not: 'tis a Pedlar's side.\n\nGREY:\nNow Warwick, speak! I say amen to avoid the\nstrangeness; exermity woman march and Galeth,\nWith givingman servan

## Ekspor Moel Generator

In [251]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [252]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
My father's rooted trie disglare
Further than way, thou out to tawn of men: he will,
For suffering 


## Kesimpulan

Dengan menggunakan  `tf.GradientTape`, model akan menghitung gradient secara otomatis untuk membantu backpropragation dengan langkah menghitung gradientnya lalu mengaplikasikan gradientnya pada model kita menggunakan `self.optimizer.apply_gradients()`.